Load the data

In [1]:
df = spark.sql("SELECT * FROM Population.BronzeTable")
display(df)

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8bdd1dc2-c5ba-47ae-8aab-04898c702060)

**Select required columns**

In [2]:
df = df.select(['Location','Time','AgeGrp','PopMale','PopFemale'])

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 4, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2cd2c2fc-1757-4d23-bf39-ca552a7a476d)

**Group by data on Age Group**

In [6]:
df_Agegrp = df.groupBy('AgeGrp').agg({"AgeGrp":"count"})
display(df_Agegrp)

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1e6d0cf2-78f9-47e3-8281-09a976111c05)

**Clean the incorrect group like "09-May","9-May,"14-Oct" etc**

In [7]:
from pyspark.sql import functions as F
df = df.withColumn('AgeGrp',F.regexp_replace('AgeGrp',"09-May","5-0"))
df = df.withColumn('AgeGrp',F.regexp_replace('AgeGrp',"9-May","5-9"))
df = df.withColumn('AgeGrp',F.regexp_replace('AgeGrp',"14-Oct","10-14"))
df = df.withColumn('AgeGrp',F.regexp_replace('AgeGrp',"#N/A","NA"))

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 9, Finished, Available, Finished)

**Check the AgeGrp column groups again**

In [9]:
display(df.groupBy('AgeGrp').agg({"AgeGrp":"count"}))

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf93d7ec-b3bb-4790-801d-9490083fa38d)

**Rename the columns and change the data type using Data Wrangler tool**

In [10]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql import types as T

def clean_data(df):
    # Rename column 'PopMale' to 'Male'
    df = df.withColumnRenamed('PopMale', 'Male')
    # Rename column 'PopFemale' to 'Female'
    df = df.withColumnRenamed('PopFemale', 'Female')
    # Change column type to int64 for column: 'Time'
    df = df.withColumn('Time', df['Time'].cast(T.LongType()))
    # Change column type to float64 for column: 'Male'
    df = df.withColumn('Male', df['Male'].cast(T.DoubleType()))
    # Change column type to float64 for column: 'Female'
    df = df.withColumn('Female', df['Female'].cast(T.DoubleType()))
    return df

df_clean = clean_data(df)
display(df_clean)

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 01122a1f-6ed1-406d-9029-c604aa35d48a)

**Unpiovt the Male and female columns in Gender and Population**

In [11]:
df_clean = df_clean.unpivot(['Location','Time','AgeGrp'],['Male','Female'],"Gender","Population")

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 31, Finished, Available, Finished)

**Save this data fram in lake house with name SliverTable**

In [12]:
df_clean.write.format("delta").saveAsTable("SilverTable")

StatementMeta(, 03b3a0e4-8281-490e-82de-cdd779ea87ac, 32, Finished, Available, Finished)